In [1]:
import requests
import sys
sys.path.append('..')
from ltr.client.solr_client import SolrClient

client = SolrClient(host='http://aips-solr:8983/solr')

# Listing 10.3 

In [2]:
from ltr.judgments import Judgment

Judgment(grade=1, keywords='social network', doc_id=37799)

Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[],weight=1

# Listing 10.4

In [3]:
mini_judg_list=[
    # for 'social network' query
    Judgment(grade=1, keywords='social network', doc_id='37799'), #The Social Network
    Judgment(grade=0, keywords='social network', doc_id='267752'), # #chicagoGirl
    Judgment(grade=0, keywords='social network', doc_id='38408'), # Life As We Know It
    Judgment(grade=0, keywords='social network', doc_id='28303'), # The Cheyenne Social Club
    
    # for 'star wars' query
    Judgment(grade=1, keywords='star wars', doc_id='11'), # star wars
    Judgment(grade=1, keywords='star wars', doc_id='1892'), # return of jedi
    Judgment(grade=0, keywords='star wars', doc_id='54138'),# Star Trek Into Darkness
    Judgment(grade=0, keywords='star wars', doc_id='85783'), # The Star
    Judgment(grade=0, keywords='star wars', doc_id='325553'), # Battlestar Galactica
]

mini_judg_list

[Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=267752,features=[],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=38408,features=[],weight=1,
 Judgment(grade=0,qid=1,keywords=social network,doc_id=28303,features=[],weight=1,
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=11,features=[],weight=1,
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=1892,features=[],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=54138,features=[],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=85783,features=[],weight=1,
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=325553,features=[],weight=1]

# Listing 10.5

In [4]:
mini_judg_list[0].features


[]

In [5]:
requests.delete('http://aips-solr:8983/solr/tmdb/schema/feature-store/movies')

import requests

feature_set = [
    {
      "name" : "title_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    {
      "name" : "release_year",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}release_year"
}}]

resp = requests.put('http://aips-solr:8983/solr/tmdb/schema/feature-store',
                    json=feature_set)
resp.text

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":25}}\n'

In [7]:
import requests

logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"social network\"]",
    'q': "id:37799 OR id:267752 id:38408 OR id:28303", #social network graded documents
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://aips-solr:8983/solr/tmdb/select',
                     data=logging_solr_query)

resp.json()

{'responseHeader': {'zkConnected': True,
  'status': 0,
  'QTime': 112,
  'params': {'q': 'id:37799 OR id:267752 id:38408 OR id:28303',
   'fl': 'id,title,[features store=movies efi.keywords="social network"]',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 4,
  'start': 0,
  'docs': [{'id': '28303',
    'title': 'The Cheyenne Social Club',
    '[features]': 'title_bm25=3.4286602,overview_bm25=3.108672,release_year=1970.0'},
   {'id': '267752',
    'title': '#chicagoGirl',
    '[features]': 'title_bm25=0.0,overview_bm25=6.0172443,release_year=2013.0'},
   {'id': '38408',
    'title': 'Life As We Know It',
    '[features]': 'title_bm25=0.0,overview_bm25=4.353118,release_year=2010.0'},
   {'id': '37799',
    'title': 'The Social Network',
    '[features]': 'title_bm25=8.243603,overview_bm25=3.814361,release_year=2010.0'}]}}